In [ ]:
%matplotlib inline
import sys
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import scipy as sp

# Data reading
import pandas as pd
import csv
import pickle

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# networkx

import networkx as nx
from networkx.algorithms import community
from networkx.algorithms.community import greedy_modularity_communities
from networkx.algorithms.community import k_clique_communities

from community import community_louvain

import scipy.sparse.linalg

### Protein-protein interaction networks

The protein-protein interaction network of Yeast *saccharomyces cerevisiae* has more than $6,000$ proteins. Proteins form physical complexes of $5$-$50$ proteins and there are roughly $100$-$200$ functional communities. 

The gene HST1 (*standard name*) is also called YOL068C (*systematic name*)

See https://www.yeastgenome.org for details; the data can be dowloaded from https://string-db.org/cgi/network.pl?taskId=nhiAd00Ggrzi

Thanks to Eduardo Altmann for sharing his Jupyter notebooks from which we have shamelessly copied

# I. Read in networks 

In [ ]:
# Read networks from file
#G_BDF1=nx.read_weighted_edgelist("./../Data/4932_protein_links_v11_0.txt",comments="#",nodetype=str)
G_BDF1=nx.read_weighted_edgelist("4932.protein.links.v11.0.txt",comments="#",nodetype=str)

print('number of nodes of G:',nx.number_of_nodes(G_BDF1))
print('number of edges of G:',nx.number_of_edges(G_BDF1))

In [ ]:
node_target = '4932.YLR399C' # BDF1 
print('BDF1 has %i links'%G_BDF1.degree(node_target))

# Choose the network to be analyzed below
G0=G_BDF1

highest_degree_node = max(G0.nodes, key=G0.degree)
print("The node with the maximum degree is: ", highest_degree_node,
      ". It has degree: ", G0.degree(highest_degree_node))



In [ ]:
# Choose the network to be analyzed below
G0=G_BDF1

print('number of nodes of the full G:',len(G0.nodes))
print('number of edges of the full G:',nx.number_of_edges(G0))
print('Is the full G connected?',nx.connected.is_connected(G0))
print('How many connected subgraphs are there?',nx.connected.number_connected_components(G0))

# delete those edges with a combined score of <= thershold_score (small confidence)
threshold_score = 700
#threshold_score = 0

for edge in G0.edges: 
    G0.get_edge_data(edge[0],edge[1])
    weight = list(G0.get_edge_data(edge[0],edge[1]).values())
    #print('qwe',weight[0])
    if(weight[0] <= threshold_score):
        G0.remove_edge(edge[0],edge[1])

# restrict to largest connected component
largest_cc = max(nx.connected_components(G0),key=len)
G0=G0.subgraph(largest_cc)
print('number of nodes of restricted G:',len(G0.nodes))
print('number of edges of restricted G:',nx.number_of_edges(G0))

# randomize according to degree-preserving Maslov-Sneppen algorithm
# G0_randomized = nx.random_reference(G0,connectivity=True) 


# II.  Community detection

In [ ]:
def partToList(part):
    '''Print partition as a list'''
    N=0
    for b in range(len(part)):
        N=N+len(part[b])
    listP=np.zeros(N)
    for b in range(len(part)):
        for n in part[b]:
            listP[n]=b
    return(listP)

def partToList_for_strings(part,G):
    '''Print partition as a list'''
    N=0
    for b in range(len(part)):
        N=N+len(part[b])
    listP=np.zeros(N)
    for b in range(len(part)):
        for n in part[b]:
            ind_n = list(G.nodes).index(n)
            listP[ind_n]=b
    return(listP)

## II a. Spectral methods

## Modularity

In [ ]:
G0_mod = nx.modularity_matrix(G0, nodelist=None, weight=None)

# modularity (is numpy.matrix)
EVal, EVec = np.linalg.eig(G0_mod)
idx = np.argsort(EVal)
idx = idx[::-1]
EVal_mod = EVal.real[idx]
EVec_mod = EVec[:,idx]
print('lambda_mod=',EVal_mod[1:6])

In [ ]:
partClub = np.sign(EVec_mod.real[:,0])
ind_neg = [i for i in range(len(partClub)) if partClub[i]<0] 
ind_pos = [i for i in range(len(partClub)) if partClub[i]>0] 
partClub = (ind_neg,ind_pos)

## Random walk (almost invariant sets)

In [ ]:
G0_Ad = nx.adjacency_matrix(G0, nodelist=None, weight=None)
# adjacency matrix (is scipy.sparse.csr.csr_matrix)

G0_D = G0_Ad.sum(axis=1)
G0_D = sp.sparse.spdiags(G0_D.flatten(), [0], len(G0.nodes), len(G0.nodes), format='csr')
G0_Markov = sp.sparse.linalg.inv(G0_D).dot(G0_Ad) # right eigenvector (expectation values)
G0_Markov = G0_Ad.dot(sp.sparse.linalg.inv(G0_D)) # left eigenvector (density)

# Markov matrix (is scipy.sparse.csr.csr_matrix)
G0_Markov = G0_Markov.astype(float);
EVal, EVec = sp.sparse.linalg.eigs(G0_Markov,k=6)
idx = np.argsort(EVal)
idx = idx[::-1]
EVal_Markov = EVal.real[idx]
EVec_Markov = EVec[:,idx]
print('lambda_Markov=',EVal_Markov.real)

In [ ]:
partClub = np.sign(EVec_Markov.real[:,1])
ind_neg = [i for i in range(len(partClub)) if partClub[i]<0] 
ind_pos = [i for i in range(len(partClub)) if partClub[i]>0] 
partClub = (ind_neg,ind_pos)

## Graph Laplacian

In [ ]:
G0_Lap = nx.laplacian_matrix(G0, nodelist=None, weight=None)

# graph Laplacian matrix (is scipy.sparse.csr.csr_matrix)
G0_Lap = G0_Lap.astype(float)

EVal, EVec = sp.sparse.linalg.eigs(G0_Lap,k=15,which='SR')
idx = np.argsort(EVal)
#idx = idx[::-1]
EVal_Lap = EVal.real[idx]
EVec_Lap = EVec[:,idx]
print('lambda_Lap=',EVal_Lap)

#G0_lap_v2 = nx.fiedler_vector(G0, weight=None, normalized=False, tol=1e-08, seed=None)

## compare different eigenvalue routines (don't trust the higher ones)
# e = np.linalg.eigvals(G0_Lap.A)
# L.A is shorthand for L.toarray()

plt.plot(EVal_Lap,"-d")
plt.xlabel("$B$")
plt.ylabel("$\lambda$")
plt.show()

In [ ]:
partClub = np.sign(EVec_Lap.real[:,1])

ind_neg = [i for i in range(len(partClub)) if partClub[i]<0] 
ind_pos = [i for i in range(len(partClub)) if partClub[i]>0] 
partClub = (ind_neg,ind_pos)

## Normalized Graph Laplacian

In [ ]:
G0_normLap = nx.normalized_laplacian_matrix(G0, nodelist=None, weight=None)

# normalized graph Laplacian matrix (is scipy.sparse.csr.csr_matrix)
G0_normLap = G0_normLap.astype(float)
EVal, EVec = sp.sparse.linalg.eigs(G0_normLap,k=6,which='SR')
idx = np.argsort(EVal)
#idx = idx[::-1]
EVal_normLap = EVal.real[idx]
EVec_normLap = EVec[:,idx]
print('lambda_normLap=',EVal_normLap)

G0_normlap_v2 = nx.fiedler_vector(G0, weight=None, normalized=True, tol=1e-08, seed=None)


In [ ]:
partClub = np.sign(EVec_normLap.real[:,1])

ind_neg = [i for i in range(len(partClub)) if partClub[i]<0] 
ind_pos = [i for i in range(len(partClub)) if partClub[i]>0] 
partClub = (ind_neg,ind_pos)

## Adjacency matrix

In [ ]:
G0_Ad = nx.adjacency_matrix(G0, nodelist=None, weight=None)

# adjacency matrix (is scipy.sparse.csr.csr_matrix)
G0_Ad = G0_Ad.astype(float)
EVal, EVec = sp.sparse.linalg.eigs(G0_Ad,k=6,which='LR')
idx = np.argsort(EVal)
idx = idx[::-1]
EVal_Ad = EVal.real[idx]
EVec_Ad = EVec[:,idx]
print('lambda_Ad=',EVal_Ad)

plt.plot(EVal_Ad,"-d")
plt.xlabel("$B$")
plt.ylabel("$\lambda$")
plt.show()

In [ ]:
partClub = np.sign(EVec_Ad.real[:,1])
ind_neg = [i for i in range(len(partClub)) if partClub[i]<0] 
ind_pos = [i for i in range(len(partClub)) if partClub[i]>0] 
partClub = (ind_neg,ind_pos)

## II b. Modularity community detection algorithms

### Louvain group modularity algorithm

!pip3 install python-louvain

https://iopscience.iop.org/article/10.1088/1742-5468/2008/10/P10008

https://en.wikipedia.org/wiki/Louvain_modularity


In [ ]:
partLouvain = community_louvain.best_partition(G0)
number_of_communities = max(list(partLouvain.values()))+1
print('# of partitions for Louvain modularity =',number_of_communities)

#### identify the community to which HST1 (=YOL068C) belongs to (cluster starts at 0)

In [ ]:
node_target = '4932.YOL068C' # HST1
print('The target protein HST1 belongs to community #',partLouvain[node_target])

#### find the communities to which this community connects to

In [ ]:
# define the communities as separate graphs

#subgraph dict with community number as key and subgraph as value
G_cluster = {}

#node dict with community number as key and node as value
nodes = {}
for i in range(number_of_communities):
    nodes[i] = []

for name, community in partLouvain.items():
    nodes[community].append(name)
    
for key in nodes.keys():
    G_cluster[key] = G0.subgraph(nodes[key])
    

#### find the communities which have links to the community of the target protein

In [ ]:
edges = {}
for i in range(number_of_communities-1):
    for j in range(i+1,number_of_communities):
        edges[str(i)+str(j)] = 0

for i in range(number_of_communities-1):
    for node in G_cluster[i].nodes():
        for neighbor in G0.neighbors(node):
            for j in range(i+1,number_of_communities):
                if neighbor in nodes[j]:
                    edges[str(i)+str(j)] += 1
print(edges) # convention: 'ij' denotes the edge between node i and node j

In [ ]:
# Find the communities which have links to the community of the target protein
target_cluster = partLouvain[node_target]
neighbor_cluster = []
for i in range(number_of_communities):
    if i < target_cluster:
        if edges[str(i)+str(target_cluster)] != 0:
            neighbor_cluster.append(i)
    if i > target_cluster:
        if edges[str(target_cluster)+str(i)] != 0:
            neighbor_cluster.append(i)
        
print(neighbor_cluster)


#### find the central nodes for each cluster

In [ ]:
top10_final = {}
for i in range(number_of_communities):
    top10_final[i] = []
    G = G_cluster[i]
    measuresNames=["Degree","Eigenvector","Katz","Page Rank","Betweeness"]
    if len(G.nodes)>2:
        listMeasures = [dict(nx.degree(G)),nx.eigenvector_centrality_numpy(G),nx.katz_centrality_numpy(G),nx.pagerank(G),nx.betweenness_centrality(G)]
   
    if len(G.nodes)<=2:
        listMeasures = [dict(nx.degree(G)),nx.eigenvector_centrality(G),nx.katz_centrality(G),nx.pagerank(G),nx.betweenness_centrality(G)]
   
    for idx,dictMeasure in enumerate(listMeasures):
        top10 = [[dictMeasure[k],k] for k in dictMeasure.keys()] # Choose the 10 largest values
        top10.sort(reverse=True)
        top10_final[i].append(top10)
        print("\n Centrality Measure in Cluster:",str(i),measuresNames[idx])
        for idx,pair in enumerate(top10[:10]):
            print(str(idx+1),": \t is node ",pair[1],' with value: %.4f \t' %(pair[0]))


In [ ]:
# write data file
central_nodes = {'COMMUNITY':[], 'RANK':[], 'DEGREE_NODE':[],'DEGREE_VALUE':[], 'EIGENVECTOR_NODE':[],'EIGENVECTOR_VALUE':[], 'KATZ_NODE':[], 'KATZ_VALUE':[], 'PAGERANK_NODE':[], 'PAGERANK_VALUE':[], 'BETWEENESS_NODE':[], 'BETWEENESS_VALUE':[]}
 
for i in range(number_of_communities):
    G = G_cluster[i]
    #the number of nodes in top10 central nodes
    row_number = min(len(G.nodes),10)
    central_nodes['COMMUNITY'].extend([i]*row_number)
    central_nodes['RANK'].extend(list(range(1,row_number+1)))
    
    for j in range(row_number):
        central_nodes['DEGREE_NODE'].append(top10_final[i][0][j][1][5:])
        central_nodes['DEGREE_VALUE'].append(top10_final[i][0][j][0])
        central_nodes['EIGENVECTOR_NODE'].append(top10_final[i][1][j][1][5:])
        central_nodes['EIGENVECTOR_VALUE'].append(top10_final[i][1][j][0])
        central_nodes['KATZ_NODE'].append(top10_final[i][2][j][1][5:])
        central_nodes['KATZ_VALUE'].append(top10_final[i][2][j][0])
        central_nodes['PAGERANK_NODE'].append(top10_final[i][3][j][1][5:])
        central_nodes['PAGERANK_VALUE'].append(top10_final[i][3][j][0])
        central_nodes['BETWEENESS_NODE'].append(top10_final[i][4][j][1][5:])
        central_nodes['BETWEENESS_VALUE'].append(top10_final[i][4][j][0])

df = pd.DataFrame(central_nodes, columns= ['COMMUNITY','RANK', 'DEGREE_NODE','DEGREE_VALUE', 'EIGENVECTOR_NODE','EIGENVECTOR_VALUE', 'KATZ_NODE', 'KATZ_VALUE', 'PAGERANK_NODE', 'PAGERANK_VALUE', 'BETWEENESS_NODE', 'BETWEENESS_VALUE'])
df.to_csv("HST1_Centrality_Louvain_thresh%s.csv" %threshold_score, index=False)


In [ ]:
# write data file
top10_final = {}
for i in range(number_of_communities):
    top10_final['COMMUNITY' + str(i)] = []
    G = G_cluster[i]
    measuresNames=["Degree","Eigenvector","Katz","Page Rank","Betweeness"]
    if len(G.nodes)>2:
        listMeasures = [dict(nx.degree(G)),nx.eigenvector_centrality_numpy(G),nx.katz_centrality_numpy(G),nx.pagerank(G),nx.betweenness_centrality(G)]
   
    if len(G.nodes)<=2:
        listMeasures = [dict(nx.degree(G)),nx.eigenvector_centrality(G),nx.katz_centrality(G),nx.pagerank(G),nx.betweenness_centrality(G)]
   
    for idx,dictMeasure in enumerate(listMeasures):
        top10 = [[dictMeasure[k],k] for k in dictMeasure.keys()] # Choose the 10 largest values
        top10.sort(reverse=True)
        for idx,pair in enumerate(top10[:10]):
            top10_final['COMMUNITY' + str(i)].append(pair[1])    
        if len(G.nodes)< 10:
            top10_final['COMMUNITY' + str(i)].extend(['None']*(10-len(G.nodes)))
df = pd.DataFrame(top10_final, columns= list(top10_final.keys()))
df.to_csv("HST1_Centrality_Louvain_thresh%s_singlecolumn.csv" %threshold_score, index=False)

# III. Analysis

In [ ]:
# specify the partition to be analyzed
partComm = partLouvain

## Avenue #1:
### restrict analysis to the cluster of the target node and identify the shortest path between the target node and one of the most central nodes of that same community

In [ ]:
# restrict to community of target node
comm_target = partComm[node_target]
G_av1 = G_cluster[comm_target]

#node_central = '4932.YNL031C'
node_central = '4932.YBR136W' # telomere maintenance
#node_central = '4932.YBL021C' # metabolism

shortest_path_av1 = nx.shortest_path(G_av1, source=node_target, target=node_central)
len_shortest_path_av1 = len(list(shortest_path_av1))

print('shortest path between the target protein and the specified central node:', shortest_path_av1)
#print(list(nx.all_simple_paths(G_av1, source=node_target, target=node_central, cutoff=len_shortest_path_av1+1)))


# VI. Centrality measures 

## Clustering

$\large{C_{net}}$

In [ ]:
#G=G_BDF1
G = G0
G_randomized = nx.random_reference(G,connectivity=True) 
print('C_net of G:',nx.transitivity(G))
print('C_net of randomized G:',nx.transitivity(G_randomized));

In [ ]:
cluster=nx.clustering(G,nodes=G.nodes)

In [ ]:
#print('Clustering:',nx.clustering(G))
totalC=0
cluster=nx.clustering(G,nodes=G.nodes)
cluster_keys=list(cluster)
cluster_values=list(cluster.values())
for n in range(len(G.nodes)):
    totalC = totalC+cluster_values[n]
print("Average clustering of G = "+str(totalC/nx.number_of_nodes(G)) )

totalC=0
cluster=nx.clustering(G_randomized,nodes=G_randomized.nodes)
cluster_keys=list(cluster)
cluster_values=list(cluster.values())
for n in range(len(G_randomized.nodes)):
    totalC = totalC+cluster_values[n]
print("Average clustering of randomized G= "+str(totalC/nx.number_of_nodes(G_randomized)) )


## Degree

In [ ]:
dgList=[]
#print('asd',G.degree)
for i in G.degree():
    dgList.append([i[1],i[0]])
dgList.sort()
dgList.reverse()
for j in dgList[:10]:
    print("Node "+str(j[1])+" has degree "+str(j[0])) 

## Shortest path

In [ ]:
dij=dict(nx.shortest_path_length(G))
diList=[]
for k in dij:
    di=np.average(list(dict(dij[k]).values()))
    diList.append([di,k])

diList.sort()
print("Top 10 nodes with shortest <d_i>:\n")
for i in diList[:10]:
    print("\t Node i=",i[1]," has <d_i>=",i[0])

## Betweeness centrality

In [ ]:
nx.betweenness_centrality(G);

In [ ]:
nx.betweenness_centrality(G_randomized);

## Eigenvector

In [ ]:
nx.eigenvector_centrality(G);

In [ ]:
nx.eigenvector_centrality(G_randomized);

In [ ]:
measuresNames=["Degree","Eigenvector","Katz","Page Rank","Betweeness"]
listMeasures = [dict(nx.degree(G)),nx.eigenvector_centrality_numpy(G),nx.katz_centrality_numpy(G),nx.pagerank(G),nx.betweenness_centrality(G)]

for idx,dictMeasure in enumerate(listMeasures):
    top10 = [[dictMeasure[k],k] for k in dictMeasure.keys()] # Choose the 10 largest values
    top10.sort(reverse=True)
    print("\n Centrality Measure:",measuresNames[idx])
    for idx,pair in enumerate(top10[:10]):
        print(str(idx+1),": \t is node ",pair[1]," with value:\t",pair[0])

In [ ]:
measuresNames=["Degree","Eigenvector","Katz","Page Rank","Betweeness"]
listMeasures = [dict(nx.degree(G_randomized)),nx.eigenvector_centrality_numpy(G_randomized),nx.katz_centrality_numpy(G_randomized),nx.pagerank(G_randomized),nx.betweenness_centrality(G_randomized)]

for idx,dictMeasure in enumerate(listMeasures):
    top10 = [[dictMeasure[k],k] for k in dictMeasure.keys()] # Choose the 10 largest values
    top10.sort(reverse=True)
    print("\n Centrality Measure:",measuresNames[idx])
    for idx,pair in enumerate(top10[:10]):
        print(str(idx+1),": \t is node ",pair[1]," with value:\t",pair[0])